In [34]:
import pandas as pd
import numpy as np

na_values = ["unknown"]

df = pd.read_csv('base_test_x.csv',
                 na_values=na_values,
                 sep=',')

In [35]:
ides = df['ID']

In [36]:
df.shape

(11747, 20)

In [37]:
df['TRABAJO'].fillna(df['TRABAJO'].mode()[0], inplace=True)
print("Valores perdidos en TRABAJO: " + 
      str(df['TRABAJO'].isnull().sum()))

def clas_work(x):
    if x  in ['student','unemployed','retired']:
        return 0
    else:
        return 1
    
df['TRABAJO'] = df.TRABAJO.apply(clas_work)


df['ESTADO_CIVIL'].fillna(df['ESTADO_CIVIL'].mode()[0], inplace=True)
print("Valores perdidos en ESTADO_CIVIL: " + 
      str(df['ESTADO_CIVIL'].isnull().sum()))

df = df.replace({'ESTADO_CIVIL':{' married':'married', 'marrid': 'married','singgle ':'single'
                            ,'single ': 'single', 'divorced ':'divorced','d_ivorced': 'divorced'}})


df = pd.concat([df.drop(columns='ESTADO_CIVIL', axis=1),pd.get_dummies(df['ESTADO_CIVIL'], drop_first=True)], axis=1)

df['EDUCACION'].fillna(df['EDUCACION'].mode()[0], inplace=True)
print("Valores perdidos en EDUCACION: " + 
      str(df['EDUCACION'].isnull().sum()))

df['EDUCACION'] = df.EDUCACION.replace({'illiterate': 'basic','basic.6y': 'basic','basic.4y':'basic','basic.9y':'basic','unknown':np.nan})
df['EDUCACION'] = df['EDUCACION'].replace({'basic':1,'high.school': 2 ,'professional.course':3, 'university.degree':4 })

df.drop(['INCUMPLIMIENTO'], axis=1, inplace=True)

df['VIVIENDA'].fillna(df['VIVIENDA'].mode()[0], inplace=True)
print("Valores perdidos en VIVIENDA: " + 
      str(df['VIVIENDA'].isnull().sum()))

df['VIVIENDA'] = df['VIVIENDA'].replace({'yes':1,'no':0})


# Imputamos la variable PRESTAMO con la moda
df['PRESTAMO'].fillna(df['PRESTAMO'].mode()[0], inplace=True)
print("Valores perdidos en PRESTAMO: " + 
      str(df['PRESTAMO'].isnull().sum()))

df['PRESTAMO'] = df['PRESTAMO'].replace({'yes':1,'no':0})

df['TIPO_CONTACTO'] = df.TIPO_CONTACTO.replace({'cellular':1,'telephone':0})

df['MES'] = df['MES'].replace(
    {'may': 5,
 'sep': 9,
 'oct': 10,
 'jun': 6,
 'nov': 11,
 'apr': 4,
 'aug': 8,
 'jul': 7,
 'mar': 3,
 'dec': 12})

df['RESULT_PREVIO'] = df['RESULT_PREVIO'].replace({'failure':2,'nonexistent': 1,'success':3})

df.drop(columns=['ID','MES'], axis=1, inplace=True)

Valores perdidos en TRABAJO: 0
Valores perdidos en ESTADO_CIVIL: 0
Valores perdidos en EDUCACION: 0
Valores perdidos en VIVIENDA: 0
Valores perdidos en PRESTAMO: 0


In [27]:
df.shape

(11747, 18)

In [39]:
import pickle

In [40]:
pickled_model = pickle.load(open('model.pkl', 'rb'))


In [41]:
X_test = df.copy()

In [58]:
y_pred = pickled_model.predict_proba(X_test)[:,1]


In [59]:
len(y_pred)

11747

In [62]:
df_pred = pd.concat([ides, pd.Series(y_pred)], axis=1).rename(columns={0:'ACEPTACION'})

In [64]:
df_pred.to_csv("preds.csv", header=True, index=False)